In [2]:

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import glob
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import torch.optim as optim
from torch import nn
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os
import academictorrents as at
import tarfile
import os
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torchvision.models as models


In [8]:
# Define transformations and load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transform)



transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transform)

# Split the dataset into training, validation, and testing sets
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# Create data loaders for each set
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm

# Define the CNN model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.dropout = nn.Dropout2d()
        self.fc1 = nn.Linear(64 * 12 * 12, 128)  # Adjust the input size for the fully connected layer
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, 2)
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return nn.functional.log_softmax(x, dim=1)


# Define the training function
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    train_loss = 0
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        pbar.set_description(f'Train Loss: {train_loss / (batch_idx + 1):.4f}')

# Define the testing/evaluation function
def test(model, loader, device):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.functional.nll_loss(output, target, reduction='sum').item()  # Change val_loss to test_loss
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(loader.dataset)  # Change val_loss to test_loss
    accuracy = 100. * correct / len(loader.dataset)
    return test_loss, accuracy


# Set device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create an instance of the model and move it to the device
model = Net().to(device)

# Define optimizer, loss function, and learning rate scheduler
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 5
for epoch in range(1, num_epochs + 1):
    print(f"Epoch {epoch}/{num_epochs}:")
    train(model, train_loader, optimizer, criterion, device)
    val_loss, accuracy = test(model, val_loader, device)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {accuracy:.2f}%")


Train Loss: 2.3085:   0%|          | 1/750 [00:00<02:22,  5.27it/s]

Epoch 1/5:


Train Loss: 0.0938:   0%|          | 1/750 [00:00<01:43,  7.23it/s]

Validation Loss: 0.0680, Validation Accuracy: 97.93%
Epoch 2/5:


Train Loss: 0.0345:   0%|          | 1/750 [00:00<01:41,  7.37it/s]

...

Validation Loss: 0.0471, Validation Accuracy: 98.57%
Epoch 4/5:


Train Loss: 0.0165:   0%|          | 1/750 [00:00<01:45,  7.07it/s]

Validation Loss: 0.0498, Validation Accuracy: 98.52%
Epoch 5/5:


Train Loss: 0.0278: 100%|██████████| 750/750 [01:37<00:00,  7.70it/s]


Validation Loss: 0.0409, Validation Accuracy: 98.82%


In [53]:
# Create the test loader
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Evaluate the model on the test set
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += nn.functional.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)
accuracy = 100. * correct / len(test_loader.dataset)
print(correct)

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.2f}%")


5924
Test Loss: 0.0411, Test Accuracy: 98.73%


In [29]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

# Iterate over the test loader and print the shape of the data
for data, _ in test_loader:
    print(data.shape)
    break


torch.Size([1, 1, 28, 28])


In [68]:
import numpy as np
import matplotlib.pyplot as plt

def plot_decision_boundaries(model, device, test_loader, labels):
    # Extract the input shape from the test loader
    input_shape = next(iter(test_loader))[0].shape[1:]
    print(input_shape)

    # Get the range of values for each feature dimension in the dataset
    x_min, x_max = 0, 27
    y_min, y_max = 0, 27

    # Add some padding to the range to ensure all points are included in the plot
    x_padding = 0.1 * (x_max - x_min)
    y_padding = 0.1 * (y_max - y_min)
    x_min -= x_padding
    x_max += x_padding
    y_min -= y_padding
    y_max += y_padding

    step = 0.1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, step), np.arange(y_min, y_max, step))
    grid = np.c_[xx.ravel(), yy.ravel()]
    print(len(grid))

    # Move the grid to the device
    grid_tensor = torch.tensor(grid, dtype=torch.float32).to(device)
    num_points = grid_tensor.size(0)
    
    grid_tensor = grid_tensor.view(num_points, 1, input_shape[1], input_shape[2])
    # Get the predicted class probabilities for the grid points
    model.eval()
    with torch.no_grad():
        logits = model(grid_tensor)
        probabilities = torch.softmax(logits, dim=1)
        _, predicted_classes = torch.max(probabilities, 1)
        predicted_classes = predicted_classes.cpu().numpy()

    # Reshape the predicted classes into the grid shape
    decision_boundary = predicted_classes.reshape(xx.shape)

    # Plot the decision boundaries
    plt.figure(figsize=(8, 6))
    plt.contourf(xx, yy, decision_boundary, alpha=0.8, cmap=plt.cm.Paired)
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title('Decision Boundaries')
    plt.show()


# Assuming you have the labels for your classification problem
labels_list = [x for x in range(10)]  # Adjust the range based on the number of classes

# Assuming you have trained your model and stored it in the 'model' variable

# Set the model to evaluation mode
model.eval()

# Move the model to the desired device
model.to(device)

# Create a test dataloader without specifying the batch size
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False)

# Call the function to visualize the decision boundaries
plot_decision_boundaries(model, device, test_loader, labels_list)


torch.Size([1, 28, 28])
108864


RuntimeError: shape '[108864, 1, 28, 28]' is invalid for input of size 217728

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

def plot_decision_boundaries(model, device, test_loader):
    # Extract the input shape from the test loader
    input_shape = next(iter(test_loader))[0].shape[1:]
    
    # Access the original dataset from the Subset object
    original_dataset = test_loader.dataset.dataset
    
    # Get the data from the original dataset
    data = original_dataset.data
    
    # Apply PCA to reduce the dimensionality of the input space
    pca = PCA(n_components=2)  # Set the desired number of components
    pca.fit(data.view(-1, input_shape[0] * input_shape[1] * input_shape[2]).numpy())
    reduced_data = pca.transform(data.view(-1, input_shape[0] * input_shape[1] * input_shape[2]).numpy())
    
    # Get the range of values for each feature dimension in the reduced space
    x_min, x_max = reduced_data[:, 0].min(), reduced_data[:, 0].max()
    y_min, y_max = reduced_data[:, 1].min(), reduced_data[:, 1].max()

    # Add some padding to the range to ensure all points are included in the plot
    x_padding = 0.1 * (x_max - x_min)
    y_padding = 0.1 * (y_max - y_min)
    x_min -= x_padding
    x_max += x_padding
    y_min -= y_padding
    y_max += y_padding

    step = 0.1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, step), np.arange(y_min, y_max, step))
    grid = np.c_[xx.ravel(), yy.ravel()]
    
    # Convert the grid to a tensor and move it to the device
    grid_tensor = torch.tensor(grid, dtype=torch.float32).to(device)
    
    # Get the predicted class probabilities for the grid points
    model.eval()
    with torch.no_grad():
        logits = model(grid_tensor)
        probabilities = torch.softmax(logits, dim=1)
        _, predicted_classes = torch.max(probabilities, 1)
        predicted_classes = predicted_classes.cpu().numpy()
    
    # Reshape the predicted classes into the grid shape
    decision_boundary = predicted_classes.reshape(xx.shape)
    
    # Plot the decision boundaries
    plt.figure(figsize=(8, 6))
    plt.contourf(xx, yy, decision_boundary, alpha=0.8, cmap=plt.cm.Paired)
    plt.xlabel('PC 1')
    plt.ylabel('PC 2')
    plt.title('Decision Boundaries')
    plt.show()

plot_decision_boundaries(model, device, test_loader)